In [ ]:
!git clone https://github.com/levan92/deep_sort_realtime

In [ ]:
!pip install ultralytics

In [ ]:
!pip install -r /content/deep_sort_realtime/requirements.txt

In [1]:
!mv /content/deep_sort_realtime/deep_sort_realtime/* /content/deep_sort_realtime/
!rm -r /content/deep_sort_realtime/deep_sort_realtime

In [ ]:
%cd /content/deep_sort_realtime/

In [ ]:
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO

## Define a function to detect objects with YOLO

In [4]:
def detect_vehicle(model, frame, target_classes):
    result = model(frame, verbose=False)[0]
    detections = []
    for box in result.boxes:
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        if class_id in target_classes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            w = x2 - x1
            h = y2 - y1
            if w * h > (frame.shape[1] * frame.shape[0] * 0.001):
                class_name = target_classes[class_id]
                detection = ([x1, y1, w, h], confidence, class_name)
                detections.append(detection)
    return detections

## Create a video capture object and display the video properties

Possible sources for video capture:

0 → Default webcam

1, 2, etc. → Other cameras

"video.mp4" → File path

URL → IP camera URL

In [ ]:
cap = cv2.VideoCapture('/content/road_traffic.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'Frame Resoultion: {frame_width} x {frame_height}')
print(f'Frame Rate: {int(fps)}fps')

## Initialize the YOLO model, DeepSORT tracker and target classes

In [54]:
model = YOLO('yolov8n.pt',verbose=False)
tracker = DeepSort(max_age=15,
                   n_init = 4,
                   nms_max_overlap = 0.7,
                   max_cosine_distance = 0.3,
                   )
target_classes = {2: 'car', 5: 'bus', 7: 'truck'}

## Create a writer object for saving output video

In [55]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/content/vehicle_track.mp4', fourcc, fps, (frame_width, frame_height))

## Detect and count vehicles without tracking

In [46]:
vehicle_count = 0
while True:
    ret, og_frame = cap.read()
    if not ret:
        break
    try:
        frame = og_frame.copy()
        detections = detect_vehicle(model,frame,target_classes)
        for detection in detections:
          vehicle_count+=1
          bbox, confidence, class_name = detection
          x1, y1, w, h = map(int, bbox)
          cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 0, 255), 1)
          cv2.putText(frame, class_name, (x1, y1-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
          cv2.putText(frame, f'vehicle_count: {vehicle_count}', (10,30),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)
    except Exception as e:
        print(f"A broader error occurred: {e}")
        break  # Or handle the error as needed
out.release()
cap.release()

## Detect and count vehicles with tracking

In [56]:
vehicle_count = 0
tracked_ids = set() # Keep track of counted vehicle IDs
while True:
    ret, og_frame = cap.read()
    if not ret:
        break
    try:
        frame = og_frame.copy()
        detections = detect_vehicle(model,frame,target_classes)
        tracks = tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            if track_id not in tracked_ids:  # Check if track ID is new
                vehicle_count += 1
                tracked_ids.add(track_id)  # Add track ID to the set

            bbox = track.to_ltrb()
            class_id = track.get_det_class()
            confidence = track.get_det_conf()
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
            cv2.putText(frame, f'{track_id}:{class_id}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.putText(frame, f'vehicle_count: {vehicle_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)
    except Exception as e:
        print(f"A broader error occurred: {e}")
        break  # Or handle the error as needed
out.release()
cap.release()